In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from IPython.display import display, Markdown
import matplotlib.colors as mcolors

In [ ]:
# Display the full DataFrame without truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

# G League Website

# Guards

## Traditional

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
    
    # Remove rows with specific players
    players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
    stats = stats[~stats["PLAYER"].isin(players_to_remove)]
    df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Advanced Stats

In [ ]:
# Display the full DataFrame without truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Misc

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "BLKA", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Scoring

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]        
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']        

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in raptors_row.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]        
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']        

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in raptors_row.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]        
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']        

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in raptors_row.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Usage

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Defense

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

# Clutch Stats

## Traditional

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping
            
            # Filter for opponent 905 only
            opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'AUS']

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Advanced

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})        
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for opponent 905 only
            opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'AUS']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Misc

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for opponent 905 only
            opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'AUS']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Scoring

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Usage

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Shooting

### Shooting 5ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']        
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']        
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting 8ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting by Zone

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)


else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)


else:
    print(f"Request failed with status code {response.status_code}")

## Opponent Shooting

### Shooting 5ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for opponent 905 only
            opponent_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'AUS']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting 8ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for opponent 905 only
            opponent_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'AUS']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting by Zone

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

# Forwards

## Traditional

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
    
    # Remove rows with specific players
    players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
    stats = stats[~stats["PLAYER"].isin(players_to_remove)]
    df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
    
    # Remove rows with specific players
    players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
    stats = stats[~stats["PLAYER"].isin(players_to_remove)]
    df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
    
    # Remove rows with specific players
    players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
    stats = stats[~stats["PLAYER"].isin(players_to_remove)]
    df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Advanced Stats

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Misc

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "BLKA", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "BLKA", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "BLKA", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Scoring

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Usage

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Defense

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Clutch Stats

## Traditional

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Advanced

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})        
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "G",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Misc

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Scoring

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Usage

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Shooting

### Shooting 5ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']        
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']        
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting 8ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Shooting by Zone

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)


else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)


else:
    print(f"Request failed with status code {response.status_code}")

## Opponent Shooting

### Shooting 5ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting 8ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting by Zone

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "F",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

# Centers

## Traditional

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 'MIN', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    #stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    #stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
    
    # Remove rows with specific players
    players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
    stats = stats[~stats["PLAYER"].isin(players_to_remove)]
    df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
    
    # Remove rows with specific players
    players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
    stats = stats[~stats["PLAYER"].isin(players_to_remove)]
    df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            'Unnamed: 0', "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", 'GP', 'W', 'L', 'W_PCT', 
            'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK',
            'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'PTS_RANK',
            'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
            'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
            'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
            'PF_RANK', 'PFD_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
            'WNBA_FANTASY_PTS_RANK'
        ]
            
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "PLAYER_NAME": "PLAYER",        
        "TEAM_ABBREVIATION": "TEAM",
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%",
        "PLUS_MINUS": "+/-"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col in ["BLKA", "PF", "TOV"]:
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
    
    # Remove rows with specific players
    players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
    stats = stats[~stats["PLAYER"].isin(players_to_remove)]
    df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Advanced Stats

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Advanced",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        player_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", 
            "E_OFF_RATING", "sp_work_OFF_RATING", "E_DEF_RATING", "sp_work_DEF_RATING", 
            "E_NET_RATING", "sp_work_NET_RATING", "E_TOV_PCT", "E_USG_PCT", 
            "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", 
            "FGM_PG", "FGA_PG", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", 
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", 
            "sp_work_OFF_RATING_RANK", "E_DEF_RATING_RANK", "DEF_RATING_RANK", 
            "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK", "NET_RATING_RANK", 
            "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK", 
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK",
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", 
            "PACE_RANK", "sp_work_PACE_RANK", "PIE_RANK", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK",
            "FGA_PG_RANK", "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        player_stats = player_stats.drop(columns=[col for col in columns_to_drop if col in player_stats.columns])

        # Rename columns
        player_stats = player_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%"
        })

        # Round the MIN column to one decimal point
        if "MIN" in player_stats.columns:
            player_stats["MIN"] = player_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = player_stats[player_stats["MIN"] >= 15]

        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TO RATIO", "DEFRTG"]:
                # Invert ranking for TO RATIO (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]


        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Misc

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "BLKA", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "BLKA", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Misc",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        misc_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FANTASY POINTS", "NBA_FANTASY_PTS",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK",
            "OPP_PTS_PAINT_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK",
            "NBA_FANTASY_PTS_RANK"
        ]
        
        # Drop specified columns
        misc_stats = misc_stats.drop(columns=[col for col in columns_to_drop if col in misc_stats.columns])

        # Rename columns
        misc_stats = misc_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP",
        })

# Round the MIN column to one decimal point
        if "MIN" in misc_stats.columns:
            misc_stats["MIN"] = misc_stats["MIN"].round(1)

        # Filter for miscs who play at least 15 minutes per game
        stats = misc_stats[misc_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP", "BLKA", "PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Apply RAP team filter for display
        stats = stats[stats["TEAM"] == "RAP"]
        df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Scoring

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]        
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']        

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in raptors_row.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]        
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']        

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in raptors_row.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Scoring",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        scoring_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "FGM", "FGA", "FG_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK",
            "PCT_PTS_3PT_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_PTS_PAINT_RANK", "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK",
            "PCT_UAST_3PM_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "FGM_RANK", "FGA_RANK",
            "FG_PCT_RANK"
        ]
        
        # Drop specified columns
        scoring_stats = scoring_stats.drop(columns=[col for col in columns_to_drop if col in scoring_stats.columns])

        # Rename columns
        scoring_stats = scoring_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST",
        })

        # Round the MIN column to one decimal point
        if "MIN" in scoring_stats.columns:
            scoring_stats["MIN"] = scoring_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = scoring_stats[scoring_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks
        ranks_data = {}
        df_with_ranks = stats.copy()
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]        
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']        

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in raptors_row.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
            
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Usage

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Combined",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Usage",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        usage_stats = pd.DataFrame(result_set, columns=headers)

        # Convert MIN to per-game by dividing it by GP
        if "MIN" in usage_stats.columns and "GP" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"] / usage_stats["GP"]

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "USG_PCT_RANK",
            "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK",
            "PCT_FTM_RANK", "PCT_FTA_RANK", "PCT_OREB_RANK", "PCT_DREB_RANK",
            "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK",
            "PCT_BLK_RANK", "PCT_BLKA_RANK", "PCT_PF_RANK", "PCT_PFD_RANK",
            "PCT_PTS_RANK"
        ]
        
        # Drop specified columns
        usage_stats = usage_stats.drop(columns=[col for col in columns_to_drop if col in usage_stats.columns])

        # Rename columns
        usage_stats = usage_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
        })

        # Round the MIN column to one decimal point
        if "MIN" in usage_stats.columns:
            usage_stats["MIN"] = usage_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = usage_stats[usage_stats["MIN"] >= 15]
        
        # Sort the data by most minutes played
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']
        
        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Defense

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Defense",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "TwoWay": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to a DataFrame
        defense_stats = pd.DataFrame(result_set, columns=headers)

        # Columns to drop
        columns_to_drop = [
            "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "DEF_RATING_RANK", "DREB_RANK", 
            "DREB_PCT_RANK", "PCT_DREB_RANK", "STL_RANK", "PCT_STL_RANK",
            "BLK_RANK", "PCT_BLK_RANK", "OPP_PTS_OFF_TOV_RANK", 
            "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "DEF_WS_RANK"
        ]

        # Drop specified columns
        defense_stats = defense_stats.drop(columns=[col for col in columns_to_drop if col in defense_stats.columns])

        # Rename columns
        defense_stats = defense_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "DEF_RATING": "DEF RTG",
            "DREB": "DREB",
            "DREB_PCT": "DREB%",
            "PCT_DREB": "%DREB",
            "STL": "STL",
            "PCT_STL": "STL%",
            "BLK": "BLK",
            "PCT_BLK": "%BLK",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TOV",
            "OPP_PTS_2ND_CHANCE": "OPP PTS 2ND CHANCE",
            "OPP_PTS_FB": "OPP PTS FB",
            "OPP_PTS_PAINT": "OPP PTS PAINT",
            "DEF_WS": "DEF WS"
        })

        # Round the MINUTES column to one decimal point
        if "MIN" in defense_stats.columns:
            defense_stats["MIN"] = defense_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        stats = defense_stats[defense_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEF RTG", "OPP PTS OFF TOV", "OPP PTS 2ND CHANCE", "OPP PTS FB", "OPP PTS PAINT"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter"]
        stats = stats[~stats["PLAYER"].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks["PLAYER"].isin(players_to_remove)]
        
        # Filter for players with team "RAP"
        raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply styling only to Raptors 905 players
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

# Clutch Stats

## Traditional

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]
        
        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "GP", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "W_PCT", "BLKA", "PFD",
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK", "FG3M_RANK", "FG3A_RANK",
            "FG3_PCT_RANK", "FTM_RANK", "FTA_RANK", "FT_PCT_RANK", "OREB_RANK", "DREB_RANK", "REB_RANK",
            "AST_RANK", "TOV_RANK", "STL_RANK", "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "PTS_RANK",
            "PLUS_MINUS_RANK", "NBA_FANTASY_PTS_RANK", "DD2_RANK", "TD3_RANK", "WNBA_FANTASY_PTS_RANK"
        ]
                
        # Drop specified columns
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])
        
        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "FG_PCT": "FG%",
            "FG3M": "3PM",
            "FG3M": "3PA",
            "FG3_PCT": "3P%",
            "FT_PCT": "FT%",
            "PLUS_MINUS": "+/-",
        })
        
        # Round the MINUTES column to one decimal point
        if "MIN" in clutch_stats.columns:
            clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Filter for players who play at least 15 minutes per game
        #stats = clutch_stats[clutch_stats["MIN"] >= 15]

        # Sort by Defensive Win Shares (DEF WS)
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks to numeric columns (after filtering)
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["TOV"]:
                # Invert ranking for TO RATIO and DEFRTG (lower is better)
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:
                # Higher values are better for other columns
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks  # Store ranks for color mapping
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                # Normalize rank to 0-1 scale for color mapping
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                # Map normalized rank to color
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
                # Adjust text color for readability
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Advanced

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})        
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})        
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})        
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Advanced",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)

        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "E_OFF_RATING",
            "sp_work_OFF_RATING", "sp_work_DEF_RATING", "E_DEF_RATING", "E_NET_RATING", "sp_work_NET_RATING", "BLKA", "PFD", "E_TOV_PCT",
            "E_USG_PCT", "E_PACE", "PACE_PER40", "sp_work_PACE", "POSS", "FGM", "FGA", "FGM_PG", "FGA_PG", 
            "FG_PCT", "FGM_RANK", "FGA_RANK", "FGM_PG_RANK", "FGA_PG_RANK", "FG_PCT_RANK"
            "NBA_FANTASY_PTS", "DD2", "TD3", "WNBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK",
            "W_PCT_RANK", "MIN_RANK", "E_OFF_RATING_RANK", "OFF_RATING_RANK", "sp_work_OFF_RATING_RANK",
            "E_DEF_RATING_RANK", "DEF_RATING_RANK", "sp_work_DEF_RATING_RANK", "E_NET_RATING_RANK",
            "NET_RATING_RANK", "sp_work_NET_RATING_RANK", "AST_PCT_RANK", "AST_TO_RANK", "AST_RATIO_RANK",
            "OREB_PCT_RANK", "DREB_PCT_RANK", "REB_PCT_RANK", "TM_TOV_PCT_RANK", "E_TOV_PCT_RANK", 
            "EFG_PCT_RANK", "TS_PCT_RANK", "USG_PCT_RANK", "E_USG_PCT_RANK", "E_PACE_RANK", "PACE_RANK", 
            "sp_work_PACE_RANK", "PIE_RANK", "FG_PCT_RANK"
        ]
                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "OFF_RATING": "OFFRTG",
            "DEF_RATING": "DEFRTG",
            "NET_RATING": "NETRTG",
            "AST_PCT": "AST%",
            "AST_TO": "AST/TO",
            "AST_RATIO": "AST RATIO",
            "OREB_PCT": "OREB%",
            "DREB_PCT": "DREB%",
            "REB_PCT": "REB%",
            "TM_TOV_PCT": "TO RATIO",
            "EFG_PCT": "EFG%",
            "TS_PCT": "TS%",
            "USG_PCT": "USG%",
        })

        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["DEFRTG", "TO RATIO"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Misc

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Misc",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT",
            "NBA_FANTASY_PTS", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK",
            "PTS_OFF_TOV_RANK", "PTS_2ND_CHANCE_RANK", "PTS_FB_RANK", "PTS_PAINT_RANK",
            "OPP_PTS_OFF_TOV_RANK", "OPP_PTS_2ND_CHANCE_RANK", "OPP_PTS_FB_RANK", "OPP_PTS_PAINT_RANK",
            "BLK_RANK", "BLKA_RANK", "PF_RANK", "PFD_RANK", "NBA_FANTASY_PTS_RANK"
        ]
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PTS_OFF_TOV": "PTS OFF TO",
            "PTS_2ND_CHANCE": "2ND PTS",
            "PTS_FB": "FBPS",
            "PTS_PAINT": "PITP",
            "OPP_PTS_OFF_TOV": "OPP PTS OFF TO",
            "OPP_PTS_2ND_CHANCE": "OPP 2ND PTS",
            "OPP_PTS_FB": "OPP FBPS",
            "OPP_PTS_PAINT": "OPP PITP"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["OPP PTS OFF TO", "OPP 2ND PTS", "OPP FBPS", "OPP PITP"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Scoring

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Scoring",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "GP", "W", "L", "W_PCT", "PCT_PTS_3PT_RANK", "FGM_RANK", "FGA_RANK", "FG_PCT_RANK",
            "PCT_FGA_2PT_RANK", "PCT_FGA_3PT_RANK", "PCT_PTS_2PT_RANK", "PCT_PTS_2PT_MR_RANK", "PCT_PTS_PAINT_RANK", "PCT_PTS_OFF_TOV_RANK",
            "PCT_AST_2PM_RANK", "PCT_UAST_2PM_RANK", "PCT_AST_3PM_RANK", "PCT_UAST_3PM_RANK", "PCT_PTS_FB_RANK", "PCT_PTS_FT_RANK",
            "FGM", "FGA", "FG_PCT", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK", "MIN_RANK", "PCT_AST_FGM_RANK", "PCT_UAST_FGM_RANK", "PCT_FGA_2PT_RANK",
        ]                
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "PCT_FGA_2PT": "%FGA 2PT",
            "PCT_FGA_3PT": "%FGA 3PT",
            "PCT_PTS_2PT": "%PTS 2PT",
            "PCT_PTS_2PT_MR": "%PTS 2PT-MR",
            "PCT_PTS_3PT": "%PTS 3PT",
            "PCT_PTS_FB": "%PTS FBPS",
            "PCT_PTS_FT": "%PTS FT",
            "PCT_PTS_OFF_TOV": "%PTS OFFTO",
            "PCT_PTS_PAINT": "%PTS PITP",
            "PCT_AST_2PM": "2FGM %AST",
            "PCT_UAST_2PM": "2FGM %UAST",
            "PCT_AST_3PM": "3FGM %AST",
            "PCT_UAST_3PM": "3FGM %UAST",
            "PCT_AST_FGM": "FGM %AST",
            "PCT_UAST_FGM": "FGM %UAST"
        })        
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.1f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Usage

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Combined

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Combined",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayerclutch"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Usage",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": "0",
    "Outcome": "",
    "Location": "",
    "Month": "0",
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "TeamID": "0",
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": "0",
    "ShotClockRange": "",
    "LastNGames": "0",
    "ClutchTime": "Last 5 Minutes",
    "AheadBehind": "Ahead or Behind",
    "PointDiff": "5",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and len(data["resultSets"]) > 0:
        result_set = data["resultSets"][0]["rowSet"]
        headers = data["resultSets"][0]["headers"]

        # Convert to DataFrame
        clutch_stats = pd.DataFrame(result_set, columns=headers)
        
        # Drop unnecessary columns
        columns_to_drop = [
            "GROUP_SET", "PLAYER_ID", "NICKNAME", "TEAM_ID", "AGE", "W", "L", "GP_RANK", "W_RANK", "L_RANK", "W_PCT_RANK",
            "MIN_RANK", "USG_PCT_RANK", "PCT_FGM_RANK", "PCT_FGA_RANK", "PCT_FG3M_RANK", "PCT_FG3A_RANK", "PCT_FTM_RANK", "PCT_FTA_RANK",
            "PCT_OREB_RANK", "PCT_DREB_RANK", "PCT_REB_RANK", "PCT_AST_RANK", "PCT_TOV_RANK", "PCT_STL_RANK", "PCT_BLK_RANK", "PCT_BLKA_RANK",
            "PCT_PF_RANK", "PCT_PFD_RANK", "PCT_PTS_RANK"
        ]                        
        
        clutch_stats = clutch_stats.drop(columns=[col for col in columns_to_drop if col in clutch_stats.columns])

        # Rename columns
        clutch_stats = clutch_stats.rename(columns={
            "PLAYER_NAME": "PLAYER",
            "TEAM_ABBREVIATION": "TEAM",
            "USG_PCT": "USG%",
            "PCT_FGM": "%FGM",
            "PCT_FGA": "%FGA",
            "PCT_FG3M": "%3PM",
            "PCT_FG3A": "%3PA",
            "PCT_FTM": "%FTM",
            "PCT_FTA": "%FTA",
            "PCT_OREB": "%OREB",
            "PCT_DREB": "%DREB",
            "PCT_REB": "%REB",
            "PCT_AST": "%AST",
            "PCT_TOV": "%TOV",
            "PCT_STL": "%STL",
            "PCT_BLK": "%BLK",
            "PCT_BLKA": "%BLKA",
            "PCT_PF": "%PF",
            "PCT_PFD": "%PFD",
            "PCT_PTS": "%PTS",
            "W_PCT": "W%"
        })        
        
        # Shift the W% column one position to the right
        if "W%" in clutch_stats.columns:
            columns = clutch_stats.columns.tolist()
            w_pct_index = columns.index("W%")
            if w_pct_index + 1 < len(columns):  # Ensure not to exceed column range
                columns.insert(w_pct_index + 1, columns.pop(w_pct_index))
            clutch_stats = clutch_stats[columns]
        
        # Round MIN column
        clutch_stats["MIN"] = clutch_stats["MIN"].round(1)

        # Convert all numeric columns explicitly
        for col in clutch_stats.columns:
            if col not in ["PLAYER", "TEAM"]:
                clutch_stats[col] = pd.to_numeric(clutch_stats[col], errors='coerce')

        # Sort by minutes played
        stats = clutch_stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)
        
        # Identify numeric columns for formatting, excluding "MIN"
        numeric_cols = [col for col in stats.columns if col not in ["MIN", "GP"] and stats[col].dtype in ['float64', 'int64']]

        # Add ranks and append them
        df_with_ranks = stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            if col in ["%TOV", "%PF"]:  # Lower is better
                ranks = stats[col].rank(ascending=True, method='min').astype(int)
            else:  # Higher is better
                ranks = stats[col].rank(ascending=False, method='min').astype(int)
            df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)
        
        # Ensure MIN is displayed with one decimal precision
        if "MIN" in df_with_ranks.columns:
            styled_df = styled_df.format({"MIN": "{:.0f}"})
        
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

## Shooting

### Shooting 5ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']        
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']        
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting 8ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting by Zone

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)


else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Base",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)


else:
    print(f"Request failed with status code {response.status_code}")

## Opponent Shooting

### Shooting 5ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "5ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 5 ft.", "5-9 ft.", "10-14 ft.", "15-19 ft.", "20-24 ft.", "25-29 ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting 8ft.

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "8ft Range",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude top-level columns 30-34 ft., 35-39 ft., and 40+ ft.
        valid_top_levels = ["Less Than 8 ft.", "8-16 ft.", "16-24 ft.", "24+ ft."]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            # Handle NaN ranks by placing them at the bottom
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
            # Filter for Raptors 905 only
            raptors_row = df_with_ranks[df_with_ranks[" ", 'TEAM'] == 'RAP']

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Shooting by Zone

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks

        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Style the DataFrame with formatting and ranks
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Regular Season

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

### Tip-Off Tournament

In [ ]:
import requests
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashplayershotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "MeasureType": "Opponent",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "20",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PORound": 0,
    "Outcome": "",
    "Location": "",
    "Month": 0,
    "SeasonSegment": "",
    "DateFrom": "",
    "DateTo": "",
    "OpponentTeamID": 0,
    "VsConference": "",
    "VsDivision": "",
    "TeamID": 0,
    "Conference": "",
    "Division": "",
    "GameSegment": "",
    "Period": 0,
    "ShotClockRange": "",
    "LastNGames": 0,
    "DistanceRange": "By Zone",
    "GameScope": "",
    "PlayerExperience": "",
    "PlayerPosition": "C",
    "StarterBench": "",
    "DraftYear": "",
    "DraftPick": "",
    "College": "",
    "Country": "",
    "Height": "",
    "Weight": "",
    "ISTRound": ""
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extract relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract bottom-level and top-level column names
        bottom_level = headers[1]["columnNames"]
        top_level = headers[0]["columnNames"]

        # Exclude the 'Backcourt' entry from the top-level columns
        valid_top_levels = [
            "Restricted Area", 
            "In The Paint (Non-RA)", 
            "Mid-Range", 
            "Left Corner 3", 
            "Right Corner 3", 
            "Above the Break 3", 
            "Corner 3"
        ]
        valid_top_level_indices = [i for i, name in enumerate(top_level) if name in valid_top_levels]

        # Build the MultiIndex for the columns
        general_columns = ["PLAYER_NAME", "TEAM_ABBREVIATION"]
        valid_general_columns = ["PLAYER", "TEAM"]  # Renamed columns
        valid_lower_level_indices = [bottom_level.index(col) for col in general_columns] + [
            6 + i for i in range(len(bottom_level[6:])) if (i // 3) in valid_top_level_indices
        ]

        # Select only the valid columns from the data
        filtered_bottom_level = [bottom_level[i] for i in valid_lower_level_indices]
        result_set_filtered = [[row[i] for i in valid_lower_level_indices] for row in result_set]

        # Adjust top-level column names for the valid FGM, FGA, FG% groups
        top_level = [" "] * len(general_columns) + [
            valid_top_levels[i // 3] for i in range(len(filtered_bottom_level) - len(general_columns))
        ]
        bottom_level = valid_general_columns + [
            metric for i in range(len(valid_top_levels)) for metric in ["FGM", "FGA", "FG%"]
        ]

        # Combine top and bottom levels into a MultiIndex
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, bottom_level))
        shooting_stats = pd.DataFrame(result_set_filtered, columns=multi_index)

        # Convert all numeric columns explicitly
        for col in shooting_stats.columns:
            if col not in [(" ", "PLAYER"), (" ", "TEAM")]:
                shooting_stats[col] = pd.to_numeric(shooting_stats[col], errors="coerce")

        # Re-identify numeric columns
        numeric_cols = [col for col in shooting_stats.columns if col not in [(" ", "PLAYER"), (" ", "TEAM")]]

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in shooting_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in shooting_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        shooting_stats = shooting_stats[column_order]
        
        # Add ranks and append them
        df_with_ranks = shooting_stats.copy()
        ranks_data = {}
        for col in numeric_cols:
            ranks = shooting_stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
            df_with_ranks[col] = shooting_stats[col].apply(
                lambda x: f"{x:.3f}" if "FG%" in col[1] else f"{x:.1f}"
            ) + ' (' + ranks.astype(str) + ')'
            ranks_data[col] = ranks
            
        # Apply color formatting
        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                rank = ranks_data[col].loc[row.name]
                max_rank = ranks_data[col].max()
                min_rank = ranks_data[col].min()
                norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Higher rank is green
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors
        
        # Remove rows with specific players
        players_to_remove = ["Jamal Shead", "D.J. Carton", "Ja'Kobe Walter", "Jonathan Mogbo", "Myles Burns", "Vance Jackson"]
        shooting_stats = shooting_stats[~shooting_stats[(" ", "PLAYER")].isin(players_to_remove)]
        df_with_ranks = df_with_ranks[~df_with_ranks[(" ", "PLAYER")].isin(players_to_remove)]

        # Filter for Raptors 905 only
        raptors_row = df_with_ranks[df_with_ranks[(" ", 'TEAM')] == 'RAP']

        # Style the DataFrame with formatting and ranks
        styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

# Graveyard

### Team Ranks

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# API URL
url = "https://stats.gleague.nba.com/stats/leagueLeaders"
params = {
    "LeagueID": "20",
    "PerMode": "PerGame",
    "StatCategory": "PTS",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "Scope": "S"
}

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive"
}

# Fetching the data
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extracting headers and rows
    headers = data['resultSet']['headers']
    rows = data['resultSet']['rowSet']

    # Creating a DataFrame
    stats = pd.DataFrame(rows, columns=headers)

    # Drop unnecessary columns
    columns_to_drop = ["PLAYER_ID", "RANK", "TEAM_ID", "GP"]
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]
    
    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]

    # Sort the data by most minutes played
    stats = stats.sort_values(by="MIN", ascending=False).reset_index(drop=True)

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (including handling 0.0 as the worst rank)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col == "TOV":
            # Invert ranking for TOV (lower is better)
            ranks = stats[col].rank(ascending=True, method='min', na_option='bottom').astype("Int64")
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min', na_option='bottom').astype("Int64")
        df_with_ranks[col] = stats[col].apply(
            lambda x: f"{x:.3f}" if col == "FG%" else f"{x:.1f}"
        )
        df_with_ranks[col] += stats.index.map(
            lambda idx: f" ({ranks[idx]})" if idx in ranks.index else ""
        )
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")